In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [4]:
# Step 1: Load the data
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded)))
df.head()


Saving Laptop.csv to Laptop.csv


,Unnamed: 0,Brand,Model,Series,Processor,Processor_Gen,RAM,Hard_Disk_Capacity,OS,Rating,Price
0,0,DELL,Inspiron,NaN,i3,11th,8.0,1 TB HDD,Windows 11 Home,3.7,39040
1,1,DELL,Vostro,NaN,i5,11th,8.0,1 TB HDD,Windows 10 Home,3.6,50840
2,2,ASUS,VivoBook,15,i3,10th,8.0,512 GB SSD,Windows 11 Home,4.3,37940
3,3,DELL,Inspiron,NaN,i3,11th,8.0,1 TB HDD,256 GB SSD,4.4,44440
4,4,ASUS,TUF,Gaming,i5,10th,8.0,512 GB SSD,Windows 10 Home,4.5,57940


In [5]:
# Step 2: Preprocess product data
# Ensure 'name', 'description', 'price', and 'ram' columns exist
df.fillna('', inplace=True)
products = df.to_dict(orient='records')

<ipython-input-5-c833edf5be2b>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [6]:
# Step 3: Initialize embedding model
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:

# Create required fields
df['name'] = df['Brand'].astype(str) + " " + df['Model'].astype(str) + " " + df['Series'].astype(str)
df['description'] = (
    "Processor: " + df['Processor'].astype(str) + " (" + df['Processor_Gen'].astype(str) + "), " +
    "RAM: " + df['RAM'].astype(str) + "GB, " +
    "HDD: " + df['Hard_Disk_Capacity'].astype(str) + ", " +
    "OS: " + df['OS'].astype(str)
)
df['price'] = df['Price']
df['ram'] = df['RAM']  # ensure it's in GB as a number

# Clean up
df.fillna('', inplace=True)
products = df.to_dict(orient='records')

In [8]:
from sentence_transformers import SentenceTransformer
import faiss

retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed product texts
product_texts = [f"{p['name']} {p['description']}" for p in products]
product_embeddings = retriever_model.encode(product_texts, convert_to_numpy=True)

# Create FAISS index
embedding_dim = product_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(product_embeddings)


In [9]:
def genai_product_search(query, max_price=None, min_ram=None, top_k=5):
    # 1. Embed query and search FAISS
    query_embedding = retriever_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k * 2)

    # 2. Filter results based on metadata
    retrieved = []
    for i in indices[0]:
        p = products[i]
        if (max_price is None or p.get("price", 1e9) <= max_price) and \
           (min_ram is None or p.get("ram", 0) >= min_ram):
            retrieved.append(p)
        if len(retrieved) >= top_k:
            break

    if not retrieved:
        return "❌ No products matched your criteria.", []

    # 3. Build context for products (including their details)
    context = "\n".join([
        f"{i+1}. {p['name']} - ₹{p['price']} - {p['ram']}GB RAM\n{p['description'][:300]}..."
        for i, p in enumerate(retrieved)
    ])

    # 4. Build the prompt for the LLM
    prompt = f"""You are an intelligent shopping assistant.

User's query: "{query}"

User's preferences:
- Price: Under ₹50,000
- RAM: Prefer more RAM
- Battery life: Longer battery backup is a plus
- Usage: Remote work

From the list of laptops below, recommend the **best** one and explain why it fits the user's needs.

Available products:
{context}

💡 Recommendation:"""

    return prompt, context  # Ensure you return 'context' along with prompt

# Example query
query = "Need a laptop under ₹50,000 with good RAM and battery life for remote work"
prompt, context = genai_product_search(
    query=query,
    max_price=50000,
    min_ram=8,
    top_k=5
)

print(prompt)  # Print the generated prompt


You are an intelligent shopping assistant.

User's query: "Need a laptop under ₹50,000 with good RAM and battery life for remote work"

User's preferences:
- Price: Under ₹50,000
- RAM: Prefer more RAM
- Battery life: Longer battery backup is a plus
- Usage: Remote work

From the list of laptops below, recommend the **best** one and explain why it fits the user's needs.

Available products:
1. DELL Inspiron Ryzen - ₹49240 - 8.0GB RAM
Processor: 3450U (-), RAM: 8.0GB, HDD: 1 TB HDD, OS: 256 GB SSD...

💡 Recommendation:
